In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style('whitegrid')

In [301]:
# read all the csv files

#air
air_visit_data = pd.read_csv("air_visit_data.csv", dtype=dict(air_store_id = str))
air_reservation_data = pd.read_csv("air_reserve.csv")
air_store_info = pd.read_csv("air_store_info.csv")
date_info = pd.read_csv("date_info.csv")

#hpg
hpg_reservation_data = pd.read_csv("hpg_reserve.csv")
hpg_store_info = pd.read_csv("hpg_store_info.csv")
store_id_relation = pd.read_csv("store_id_relation.csv")

# data preparation

In [302]:

#air_database = pd.merge(air_reservation_data, air_store_info, how='left', on='air_store_id')
air_reservation_data['visit_datetime'] = pd.to_datetime(air_reservation_data['visit_datetime'])
air_reservation_data['reserve_datetime'] = pd.to_datetime(air_reservation_data['reserve_datetime'])

# add one more column, the gap between reserve time and visit time
reserve_diff = (air_reservation_data.visit_datetime - air_reservation_data.reserve_datetime)
reserve_diff = reserve_diff/ np.timedelta64(1, 'h')
air_reservation_data['reserve_gap_hour'] = reserve_diff
air_reservation_data['visit_date'] = air_reservation_data.visit_datetime.dt.date

# store the data
air_reservation_data.to_csv("./preprocessed_air_reserve_info.csv", index=False)

In [303]:
air_reservation_data.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_gap_hour,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,3.0,2016-01-01
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,0.0,2016-01-01
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,0.0,2016-01-01
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,4.0,2016-01-01
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,19.0,2016-01-01


In [304]:
# merge air_visit_data and air_store_info
air_database = pd.merge(air_visit_data, air_store_info, how='left', on='air_store_id')
date_info.rename(columns={'calendar_date': 'visit_date'}, inplace=True)
date_info.visit_date = pd.to_datetime(date_info.visit_date)
air_database.visit_date = pd.to_datetime(air_database.visit_date)

# add day information
air_database = pd.merge(air_database, date_info, how='left', on='visit_date')
air_database.sort_values(by='visit_date', axis=0, inplace=True, ignore_index=True)
# store the data
air_database.to_csv("./preprocessed_air_visit_store_date_info.csv", index=False)

In [294]:
air_database.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg
0,air_fab092c35776a9b1,2016-01-01,19,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Momochi,33.581941,130.348436,Friday,1
1,air_f26f36ec4dc5adb0,2016-01-01,64,Izakaya,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549,Friday,1
2,air_d97dabf7aae60da5,2016-01-01,102,Cafe/Sweets,Tōkyō-to Shibuya-ku Jingūmae,35.669290,139.707056,Friday,1
3,air_39dccf7df20b1c6a,2016-01-01,55,Izakaya,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,Friday,1
4,air_79f528087f49df06,2016-01-01,42,Western food,Tōkyō-to Suginami-ku Asagayaminami,35.699566,139.636438,Friday,1


In [305]:
# add total reservations made for the visit date
air_reservation_count = air_reservation_data.groupby(
                                by=['air_store_id', 'visit_date'], as_index=False)['reserve_visitors'].sum()

air_reservation_count['visit_date'] = pd.to_datetime(air_reservation_count['visit_date'])

air_database = pd.merge(air_database, air_reservation_count, how='left', on=['visit_date', 'air_store_id'])
air_database.fillna(0, inplace=True)
air_database.to_csv("preprocessed_air_visit_store_date_reserve_info.csv", index=False)

In [306]:
print("No of air stores that are part of HPG database is", store_id_relation.shape[0])

No of air stores that are part of HPG database is 150


In [307]:
# Get the hpg reservation information for hpg store ids
hpg_database = pd.merge(store_id_relation, hpg_reservation_data, on='hpg_store_id')
hpg_database.visit_datetime = pd.to_datetime(hpg_database.visit_datetime)
hpg_database.reserve_datetime = pd.to_datetime(hpg_database.reserve_datetime)
hpg_diff = hpg_database.visit_datetime - hpg_database.reserve_datetime
hpg_diff = hpg_diff / np.timedelta64(1, 'h')

hpg_database["hpg_reserve_gap_hour"] = hpg_diff
hpg_database['visit_date'] = hpg_database.visit_datetime.dt.date

In [308]:
hpg_database.head()

,air_store_id,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_reserve_gap_hour,visit_date
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,2016-01-04 12:00:00,2016-01-03 14:00:00,7,22.0,2016-01-04
1,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,2016-01-04 14:00:00,2016-01-02 13:00:00,4,49.0,2016-01-04
2,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,2016-01-05 12:00:00,2016-01-01 08:00:00,3,100.0,2016-01-05
3,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,2016-01-09 12:00:00,2016-01-07 20:00:00,6,40.0,2016-01-09
4,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,2016-01-11 12:00:00,2016-01-10 15:00:00,3,21.0,2016-01-11


In [309]:
# add total reservations made for the visit date through hpg database
hpg_reservation_count = hpg_database.groupby(
                                by=['air_store_id', 'visit_date'], as_index=False)['reserve_visitors'].sum()

hpg_reservation_count['visit_date'] = pd.to_datetime(hpg_reservation_count['visit_date'])

air_database = pd.merge(air_database, hpg_reservation_count, 
                        how='left', on=['visit_date','air_store_id'])
air_database.fillna(0, inplace=True)
air_database.rename(columns={"reserve_visitors_x": "air_reserve_visitors", 
                             "reserve_visitors_y": "hpg_reserve_visitors"},
                   inplace=True)
air_database.to_csv("preprocessed_final.csv", index=False)

In [310]:
air_database.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg,air_reserve_visitors,hpg_reserve_visitors
0,air_fab092c35776a9b1,2016-01-01,19,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Momochi,33.581941,130.348436,Friday,1,0.0,0.0
1,air_f26f36ec4dc5adb0,2016-01-01,64,Izakaya,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549,Friday,1,0.0,0.0
2,air_d97dabf7aae60da5,2016-01-01,102,Cafe/Sweets,Tōkyō-to Shibuya-ku Jingūmae,35.669290,139.707056,Friday,1,0.0,0.0
3,air_39dccf7df20b1c6a,2016-01-01,55,Izakaya,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,Friday,1,0.0,0.0
4,air_79f528087f49df06,2016-01-01,42,Western food,Tōkyō-to Suginami-ku Asagayaminami,35.699566,139.636438,Friday,1,0.0,0.0


In [314]:
# test_data preparation
submission = pd.read_csv("./sample_submission.csv")
submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [320]:
submission['air_store_id'] = submission.id.str[:20]
submission['date'] = submission.id.str[21:]
submission['date'] = pd.to_datetime(submission.date)
#submission['date'] = submission.date.dt.strftime("%d-%m-%Y")
submission.to_csv("processed_submission.csv", index=False)

## data information

In [326]:
print()
print("Train Data information".center(50, "="))
print()
print("Train dataset size: ", air_database.shape)
print("Total unique restaurants", len(air_database.air_store_id.unique()))
print("Total unique restaurant genre: ", len(air_database.air_genre_name.unique()))
print("Average visitors:", air_database.visitors.mean())
print("No of days data:", len(air_database.visit_date.unique()))

print()
print("Test Data information".center(50, "="))
print()
print("Total unique restarants: ", len(submission.air_store_id.unique()))
print("No of days to be predicted: ",len(submission.date.unique()))



==============Train Data information==============

Train dataset size:  (252108, 11)
Total unique restaurants 829
Total unique restaurant genre:  14
Average visitors 20.973761245180636
No of days data 478

==============Test Data information===============

Total unique restarants:  821
No of days to be predicted:  39
